In [1]:
#!/home/selim/DS/python/anaconda3/bin/python
from __future__ import print_function

from flask import Flask, request, send_from_directory, redirect, render_template, flash, url_for, jsonify, make_response, abort
from flask import Flask, render_template
from flask_restful import Resource, Api
import pickle
import numpy as np
import sys, os
import keras
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
%matplotlib inline
from markupsafe import Markup
from IPython.display import Image, display
from io import BytesIO
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
from gensim import models
import base64

import matplotlib.font_manager

#for tables in Jupyter
from IPython.display import HTML, display
#import tabulate
import urllib.parse
#for visualization
import math
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as sPCA
from sklearn import manifold 
import os
from os.path import join, exists
import sys
import numpy as np
import time
#from pandas.lib import TimeValidator
import re
import string
#import urllib2
import json
from zipfile import ZipFile
from gensim.models import KeyedVectors
import sys
import codecs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE

from sklearn.manifold import TSNE
import pandas as pd
import matplotlib
import graphviz
#from ggplot import *

import tensorflow as tf

from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.data_utils import get_file
from keras.utils import to_categorical, plot_model
from keras.models import Sequential, Model, load_model
from keras.models import model_from_config, model_from_json

from keras.layers import Embedding, Dense, Input, Dropout, Reshape, BatchNormalization, TimeDistributed, Lambda, Layer, merge, LSTM, Bidirectional, Convolution1D, GRU, add, concatenate
from keras.callbacks import Callback, ModelCheckpoint, TensorBoard, BaseLogger, ReduceLROnPlateau
from keras.optimizers import RMSprop, Adam, SGD, Adagrad
import sys 
from keras.utils import CustomObjectScope

from codecs import unicode_escape_encode, unicode_escape_decode, raw_unicode_escape_encode,raw_unicode_escape_decode
import logging
#sys.setrecursionlimit(2960)

Using TensorFlow backend.
C:\Users\hp\Anaconda3\envs\new\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\Anaconda3\envs\new\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\Anaconda3\envs\new\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hp\Anaconda3\envs\new\lib\site-packages\tensorflow\py

In [2]:
dataset_filename = 'multinli_0.9'
training_sample_size = 1000 # Change this to -1 if you want all
train_filename = ('D:/ett.json')
if exists(train_filename):
    print('Extract %s' % train_filename)
    zipfile = open(train_filename, 'r', encoding = 'utf-8-sig') 

Extract D:/ett.json


In [3]:
import codecs
import io
from json import loads

def loadDataset(filename, size=-1):
    label_category = {
        'neutral': 0,
        'entailment': 1,
        'contradiction': 2
    }
    dataset = []
    sentence1 = []
    sentence2 = []
    labels = []
    with codecs.open(filename, 'r',  'utf-8-sig') as f:
        if len(f.readlines(f)) != 0:
            f.seek(0)
            i = 0
         
            for line in f:
                row = json.loads(str(line).encode())
                if size == -1 or i < size:
                    dataset.append(row)
                    label = row["gold"]
                    if label in label_category:
                        sentence1.append( row["sentence1"])
                        sentence2.append( row["sentence2"])

                        labels.append( label_category[label] )
                   
                
        return (dataset, sentence1, sentence2, labels)


(train_dataset, train_sentence1, train_sentence2, train_labels) = loadDataset(train_filename, training_sample_size)

train_df = pd.DataFrame(train_dataset)

train_df.head()

,gold,sentence1,sentence2
0,neutral,የቤተሰብ ቀን በባሕሩ ዳርቻ ላይ,ምንም እንኳን የቤተሰብ ቀን ቢሆንም ብዙ ሰዎች በባሕሩ ዳርቻ ላይ አሉ
1,contradiction,ፖሊሱ ሰውየውን መታው,ሴትየዋ የተመታችው በፖሊሱ ነው
2,entailment,ፖሊሱ ሰውየውን መታው,ሰውየው የተመታው በፖሊሱ ነው
3,entailment,አንድ ሰው ስልኩን እያየ ነው,ሰውየው ስልኩን እየተመለከተ ነው
4,neutral,አንድ ሰው ስልኩን እየነካካ ነው,ሰውየው ስልኩን እየተመለከተ ነው


In [4]:
print(len(train_dataset))

30


In [5]:
train_df_features = train_df[['sentence1', 'sentence2', 'gold']]


In [6]:
IMAGE_FOLDER = os.path.join('static', 'img_pool')

In [7]:
w2v= None
sw = None
graph =None



In [8]:
pathft = 'E:/ftnfl.txt'
pathwv = 'E:/wvnfl.txt'

In [9]:
app = Flask(__name__)
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'
app.config['UPLOAD_FOLDER'] = IMAGE_FOLDER

In [10]:
def ma():
    global w2v,sw
    sw = KeyedVectors.load_word2vec_format(pathft)
    w2v = KeyedVectors.load_word2vec_format(pathwv)

    graph  = tf.get_default_graph()

In [11]:
load = ma()

In [12]:
%run model_load.ipynb
__name__ == '__main__' and '__file__' not in globals()

True

In [13]:

ft = ft()
wv = wv()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [14]:
def main():
    ft
    wv
    load

  

In [15]:
%run each_plt.ipynb
__name__ == '__main__' and '__file__' not in globals()

ERROR:root:File `'each_plt.ipynb.py'` not found.


True

In [16]:
@app.route('/disimilarwv')
def dis():
    return render_template('dissimilar.html')

In [17]:
@app.route('/morphemewv')
def morp():
    return render_template('morp.html')

In [18]:
@app.route('/simwv')
def simi():
    return render_template('similar.html')

In [19]:
@app.route('/analogyw')
def ana():
    return render_template('analogy.html')

In [20]:
@app.route('/morphemew')
def swsim():
    return render_template('swsim.html')

In [21]:
@app.route('/wpca')
def pll():
    return render_template('pca.html')

In [22]:
@app.route('/simwv', methods = ["POST", "GET"])

def simwv():
    if request.method=='POST':
        if 'sentences' not in request.form :
            flash('No word post')
            redirect(request.url)
        elif request.form['sentences'] == '':
            flash('No word')
            redirect(request.url)
        else:
            texta = request.form.get('sentences')
            try:
                results = w2v.most_similar(texta.split())
            except:
                results = 'Try Again'
      
            logging.info("dissimilar for %s: %s" % (texta, results))

    return render_template('similar.html', sentences=texta,results=results) 

In [23]:
@app.route('/simft', methods = ["POST", "GET"])

def simft():
    if request.method=='POST':
        if 'sentence' not in request.form :
            flash('No word post')
            redirect(request.url)
        elif request.form['sentence'] == '':
            flash('No word')
            redirect(request.url)
        else:
            texta = request.form.get('sentence')
            try:
        

                results = sw.most_similar(texta.split())
            except:
                results='Try Again'
      
                results.append((results))
                results=','.join(str(string) for string in results)
                #new_list = codecs.decode(new_list, "unicode_escape",errors="replace")
                results = '<br>'.join(str(string) for string in results)


    return render_template('similar.html', sentence=texta,result=results) 

In [24]:
@app.route('/disimilarwv', methods = ["POST", "GET"])

def disimilarwv():
    if request.method=='POST':
        if 'sentence' not in request.form :
            flash('Post at least three words')
            redirect(request.url)
        elif request.form['sentence'] == '':
            flash('No words')
            redirect(request.url)
        else:
            texta = request.form.get('sentence')
            try:
                resultd = w2v.wv.doesnt_match(texta.split())
            except:
                resultd = 'Try Again'
            logging.info("dissimilar for %s: %s" % (texta, resultd))
            y= ("dissimilar for %s: %s" % (texta, resultd))
          
    return render_template('dissimilar.html', y= y, sentence=texta, result=resultd) 

In [25]:
@app.route('/disimilarft', methods = ["POST", "GET"])

def disimilarft():
    if request.method=='POST':
        if 'sentenced' not in request.form :
            flash('Post at least three words')
            redirect(request.url)
        elif request.form['sentenced'] == '':
            flash('No words')
            redirect(request.url)
        else:
            texta = request.form.get('sentenced')
            try:
                resultd = sw.wv.doesnt_match(texta.split())
            except:
                resultd = 'Try Again'
            logging.info("dissimilar for %s: %s" % (texta, resultd))
            x= ("The most dissimilar for query %s: is  %s" % (texta, resultd))
          
    return render_template('dissimilar.html',x=x ,sentenced=texta, resultd=resultd) 

In [26]:
@app.route('/morphemewv', methods = ["POST", "GET"])

def morphemewv():
    if request.method=='POST':
        if 'sentencew' not in request.form :
            flash('No sentence post')
            redirect(request.url)
        elif request.form['sentencew'] == '':
            flash('No sentence')
            redirect(request.url)
        else:
            texta = request.form.get('sentencew')
            try:
                resultw =  {search_term:[item[0] for item in w2v.wv.most_similar(texta.split(), topn=6)]
                        for search_term in texta.split()}
            except:
                resultw = 'Try Again'
      
            logging.info("MorphemsWV for %s: %s" % (texta, resultw))
       
         
    return render_template('morp.html', sentencew=texta, result=resultw) 

In [27]:
@app.route('/morphemeft', methods = ["POST", "GET"])

def morphemft():
    if request.method=='POST':
        if 'sentencef' not in request.form :
            flash('No sentence post')
            redirect(request.url)
        elif request.form['sentencef'] == '':
            flash('No sentence')
            redirect(request.url)
        else:
            texta = request.form.get('sentencef')
            try:
                resultf =  {search_term:[item[0] for item in sw.wv.most_similar(texta.split(), topn=6)]
                        for search_term in texta.split()}
            except:
                resultf = 'Try Again'
      
            logging.info("MorphemsSW for %s: %s" % (texta, resultf))
       
         
    return render_template('morp.html', sentencef=texta, resultf=resultf) 

In [28]:
@app.route('/morphemew', methods = ["POST", "GET"])

def morphemew():
    if request.method=='POST':
        if 'sentencew' not in request.form :
            flash('No sentence post')
            redirect(request.url)
        elif request.form['sentencew'] == '':
            flash('No sentence')
            redirect(request.url)
        else:
            texta = request.form.get('sentencew')
            textb = request.form.get('sentencey')
            try:
                resultt = w2v.similarity(texta,textb)
            except:
                resultt='Try Again'
      

            logging.info("Similarity score  for %s %s: %s" % (texta,textb,  resultt))
            y= ("'%s' and '%s' is %s" %  (texta,textb,  resultt))

        

         
    return render_template('swsim.html',y=y, sentencew=texta,sentencey=textb, result=resultt) 

In [29]:
@app.route('/morphemes', methods = ["POST", "GET"])

def morphemes():
    if request.method=='POST':
        if 'sentences' not in request.form :
            flash('No sentence post')
            redirect(request.url)
        elif request.form['sentences'] == '':
            flash('No sentence')
            redirect(request.url)
        else:
            texti = request.form.get('sentences')
            textj = request.form.get('sentencei')
            try:
                result = sw.similarity(texti,textj)
            except:
                result='Try Again'
      

            logging.info("Similarity score  for %s %s: %s" % (texti,textj,  result))
            x= ("'%s' and '%s' is %s" %  (texti,textj,  result))


         
    return render_template('swsim.html', x=x, sentences=texti,sentencei=textj, res=result) 

In [30]:
@app.route('/analogyw', methods = ["POST", "GET"])

def analogyw():
    if request.method=='POST':
        if 'sentencev' not in request.form :
            flash('No sentence post')
            redirect(request.url)
        elif request.form['sentencev'] == '':
            flash('No sentence')
            redirect(request.url)
        else:
            texta = request.form['sentencev']
            texta = [texta] if type(texta) == np.str else texta
            for example in texta:
                    a, b, x = example.split()
            try:
                
        
                    predicted = w2v.most_similar([x, b], [a],topn=3)
                
            except:
                predicted='Try Again'
            logging.info ("'%s' is to '%s' as '%s' is to '%s'" % (a, b, x, predicted))
            y = ("if '%s' is to '%s' as '%s' is to '%s'" % (a, b, x, predicted))


         
       
         
    return render_template('analogy.html',y=y, sentencev=texta, res= predicted) 

In [31]:
@app.route('/analogys', methods = ["POST", "GET"])

def analogys():
    if request.method=='POST':
        if 'sentences' not in request.form :
            flash('No sentence post')
            redirect(request.url)
        elif request.form['sentences'] == '':
            flash('No sentence')
            redirect(request.url)
        else:
            texta = request.form['sentences']
            texta = [texta] if type(texta) == np.str else texta
            for example in texta:
                    a, b, x = example.split()
            try:
                
        
                    predicted = sw.most_similar([x, b], [a],topn=3)
                
            except:
                predicted='Try Again'
            logging.info ("'%s' is to '%s' as '%s' is to '%s'" % (a, b, x, predicted))
            x = ("if '%s' is to '%s' as '%s' is to '%s'" % (a, b, x, predicted))


         
       
         
    return render_template('analogy.html',x =x, sentences=texta, result= predicted)

In [32]:
#PCA for word vectors 
@app.route('/wpca', methods = ["POST", "GET"])


def show_closest_line():
    if request.method=='POST':
        if 'sentence' not in request.form :
            flash('No sentence post')
            redirect(request.url)
        elif request.form['sentence'] == '':
            flash('No sentence')
            redirect(request.url)
        else:
            font = matplotlib.font_manager.FontProperties(fname = 'C:/Users/hp/Anaconda3/envs/deeg/Library/fonts/abyssinica_sil.ttf').get_name()    
            FONT_SIZE = 20
            TEXT_KW = dict(fontsize=FONT_SIZE, fontweight='bold', fontproperties=font)

            texta = request.form.get('sentence')

            n=10
            tops = w2v.similar_by_word(texta, topn=n, restrict_vocab=None)

            
            items = [texta] + [x[0] for x in tops]

            wvecs = np.array([w2v.word_vec(wd, use_norm=True) for wd in items])



            tsne = manifold.TSNE(n_components=2, init='pca')
            coords = tsne.fit_transform(wvecs)
            #print("kl-divergence: %0.8f" % tsne.kl_divergence_)




            plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
            plt.tick_params(
                axis='both',          
                which='both',      
                bottom=False,      
                left=False,         
                labelbottom=False,
                labelleft=False)

            lim = max([abs(x) for x in coords[:,0] + coords[:,1]])
            plt.xlim([-lim,lim])
            plt.ylim([-lim,lim])
            plt.scatter(coords[2:,0], coords[2:,1])
            plt.scatter(coords[0:1,0], coords[0:1,1], color='black')
            plt.scatter(coords[1:2,0], coords[1:2,1], color='orange')

            for item, x, y in zip(items[2:], coords[2:,0], coords[2:,1]):
                plt.annotate( item, xy=(x, y), xytext=(-2, 2), textcoords='offset points', 
                             ha='right', va='bottom', color='purple', **TEXT_KW)

            x0=coords[0,0]
            y0=coords[0,1]
            plt.annotate( texta , xy=(x0, y0), xytext=(-2, 2), textcoords='offset points', 
                         ha='right', va='bottom', color='black',**TEXT_KW)

            x1=coords[1,0]
            y1=coords[1,1] 
            plt.annotate( items[1] , xy=(x1, y1), xytext=(-2, 2), textcoords='offset points', 
                         ha='right', va='bottom', color='orange', **TEXT_KW )

            ax = plt.gca()

            r = math.sqrt( (x1-x0)**2 + (y1-y0)**2 )

            circle = plt.Circle((x0, y0), r, color='orange', fill=False)
            ax.add_artist(circle)

           


            img = io.BytesIO() 
            plt.savefig(img, format='png')
            img.seek(0)

            plot_url = urllib.parse.quote(base64.b64encode(img.read()).decode())

          
           # return render_template('pca.html',sentence=texta, plot_url=plot_url)
            return render_template('pca.html',sentence=texta, plot_url=plot_url)


    

In [33]:
#PCA for word vectors 
@app.route('/fpca', methods = ["POST", "GET"])


def show_closest():
    if request.method=='POST':
        if 'sentenceb' not in request.form :
            flash('No sentence post')
            redirect(request.url)
        elif request.form['sentenceb'] == '':
            flash('No sentence')
            redirect(request.url)
        else:
            font = matplotlib.font_manager.FontProperties(fname = 'C:/Users/hp/Anaconda3/envs/deeg/Library/fonts/abyssinica_sil.ttf').get_name()    
            FONT_SIZE = 20
            TEXT_KW = dict(fontsize=FONT_SIZE, fontweight='bold', fontproperties=font)

            texta = request.form.get('sentenceb')

            n=10
            tops = sw.similar_by_word(texta, topn=n, restrict_vocab=None)

            
            items = [texta] + [x[0] for x in tops]

            wvecs = np.array([w2v.word_vec(wd, use_norm=True) for wd in items])



            tsne = manifold.TSNE(n_components=2, init='pca')
            coords = tsne.fit_transform(wvecs)
            #print("kl-divergence: %0.8f" % tsne.kl_divergence_)




            plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
            plt.tick_params(
                axis='both',          
                which='both',      
                bottom=False,      
                left=False,         
                labelbottom=False,
                labelleft=False)

            lim = max([abs(x) for x in coords[:,0] + coords[:,1]])
            plt.xlim([-lim,lim])
            plt.ylim([-lim,lim])
            plt.scatter(coords[2:,0], coords[2:,1])
            plt.scatter(coords[0:1,0], coords[0:1,1], color='black')
            plt.scatter(coords[1:2,0], coords[1:2,1], color='orange')

            for item, x, y in zip(items[2:], coords[2:,0], coords[2:,1]):
                plt.annotate( item, xy=(x, y), xytext=(-2, 2), textcoords='offset points', 
                             ha='right', va='bottom', color='purple', **TEXT_KW)

            x0=coords[0,0]
            y0=coords[0,1]
            plt.annotate( texta , xy=(x0, y0), xytext=(-2, 2), textcoords='offset points', 
                         ha='right', va='bottom', color='black',**TEXT_KW)

            x1=coords[1,0]
            y1=coords[1,1] 
            plt.annotate( items[1] , xy=(x1, y1), xytext=(-2, 2), textcoords='offset points', 
                         ha='right', va='bottom', color='orange', **TEXT_KW )

            ax = plt.gca()

            r = math.sqrt( (x1-x0)**2 + (y1-y0)**2 )

            circle = plt.Circle((x0, y0), r, color='orange', fill=False)
            ax.add_artist(circle)

           


            img = io.BytesIO() 
            plt.savefig(img, format='png')
            img.seek(0)

            plot_url = urllib.parse.quote(base64.b64encode(img.read()).decode())

          
            return render_template('pca.html',sentenceb=texta, plot_url=plot_url)
    #return render_template('pca.html',sentenceb=texta, plot=plot_url)

                

 


    

In [34]:
@app.route('/', methods = ["POST", "GET"])
def index():
   

   
        
    return render_template('indexx.html')


In [35]:
@app.route('/layout', methods = ["POST", "GET"])
def home():
    return render_template('layout.html')

In [36]:
@app.route('/grid', methods =["POST", "GET"])
def grid():

   

    pltal = plotallv
    accuracyw = historyv['acc'][4]

    return render_template('grid.html',pltal=pltal,accuracyw=accuracyw )




In [37]:
@app.route('/grida', methods =["POST", "GET"])

def grida():

    pltalf=plotallf
    accuracyf = historyf['acc'][4]


    return render_template('grida.html', pltalf=pltalf,accuracyf=accuracyf)


    

In [38]:
@app.route('/plot')
def plot():
    img_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'training-history-comparison.png')
    return render_template('plot.html', img_filename=img_filename)
    

In [39]:
def prepare_datapoint(texta, textb):
    texta = [texta] if type(texta) == np.str else texta
    textb = [textb] if type(textb) == np.str else textb

    tokenizer = configf['tokenizer']['tokenizer']
    texta = tokenizer.texts_to_sequences(texta)
    textb = tokenizer.texts_to_sequences(textb)

    texta = pad_sequences(texta, maxlen = configf['tokenizer']['max_seq_length'])
    textb = pad_sequences(textb, maxlen = configf['tokenizer']['max_seq_length'])
    return(texta,textb)
            

In [40]:
def predict_entft(texta, textb):
    ret = dict()
    texta, textb = prepare_datapoint(texta,textb)
    with graphf.as_default():
            ent_class = modelf.predict([texta,textb], batch_size=1)[0]
    argmax_ent = np.argmax(ent_class)
    entel_class_txt  = (["Neutral", "Postive", "Negative"][np.argmax(ent_class)]+" entailment" )
    ent_score = ent_class[argmax_ent]
    return(entel_class_txt, ent_score)

In [41]:
def predict_entwv(texta, textb):
    ret = dict()
    texta, textb = prepare_datapoint(texta,textb)
    with graphv.as_default():
            ent_class = modelv.predict([texta,textb], batch_size=1)[0]
    argmax_ent = np.argmax(ent_class)
    entel_class_txt  = (["Neutral", "Postive", "Negative"][np.argmax(ent_class)]+" entailment" )
    ent_score = ent_class[argmax_ent]
    return(entel_class_txt, ent_score)

In [42]:
def predict_entwv(texta, textb):
    ret = dict()
    texta, textb = prepare_datapoint(texta,textb)
    with graphv.as_default():
            ent_class = modelv.predict([texta,textb], batch_size=1)[0]
    argmax_ent = np.argmax(ent_class)
    entel_class_txt  = (["Neutral", "Postive", "Negative"][np.argmax(ent_class)]+" entailment" )
    ent_score = ent_class[argmax_ent]
    return(entel_class_txt, ent_score)

In [43]:
@app.route('/ft', methods = ["POST", "GET"])

def ft():
    sone=[]
    sonemis=[]

    new_list =[]
    new_listmis =[]

    counts=0
    countf=0


   


    for (i) in range(30):
        
        trains1, trains2, label = train_df_features.iloc[i]
        sentence = trains1
        sentencea = trains2
       

        trains1, trains2 =prepare_datapoint(trains1,trains2)
      
        
        with graphf.as_default(): 
            pridiction = modelf.predict([trains1, trains2])[0]
        x =np.argmax(pridiction)
        r= (["neutral", "entailment", "contradiction"][np.argmax(pridiction)])
   
       
      
     
        
        if label==r:
            sone.append('sentence1: ' +  (sentence))
            sone.append('sentence2:  '+  (sentencea))
            sone.append('actual_label: '+(label))
            sone.append('pridicted_label: '+ r)
            counts+=1
        if label!=r:
            sonemis.append('sentence1: ' +  (sentence))
            sonemis.append('sentence2:  '+  (sentencea))
            sonemis.append('actual_label: '+(label))
            sonemis.append('pridicted_label: '+ r)
            countf+=1
                
        #new_list.append= [str(sone).split(maxsplit=0)]
        #print(len(sone))

        #new_list=[re.sub(',/n ', ' ', s.strip()) for s in sone]
        #new_list='[' + ' '(i for item in sone)+ ']'
        #new_list.append([string.replace((""","""), """ """) for string in sone])
        new_list=','.join(str(string) for string in sone)
        #new_list = codecs.decode(new_list, "unicode_escape",errors="replace")
        new_list = '<br>'.join(str(string) for string in sone)
        new_listmis=','.join(str(string) for string in sonemis)
        #new_list = codecs.decode(new_list, "unicode_escape",errors="replace")
        new_listmis = '<br>'.join(str(string) for string in sonemis)


        


        #t =(train_df_features.iloc[i][0])
        #train_df.append(pd.DataFrame.iloc[i])
        #sum=0
        #e = sum([i for z in label[i] if z==r[i]])

            
            
       
            
        
     



    #print(new_list)
    #print(new_listmis)

    #print(sone)
    #print(sonemis)





    
    if request.method=='POST':
        if not request.form["sentencea"]:
            flash('No Premises post')
            redirect(request.url)
        elif not request.form["sentenceb"]:
            flash('No Hypotesis')
            redirect(request.url)
        elif request.form['sentencea'] == '' and  request.form['sentenceb'] == '':
            flash('Please Enter the two Sentences')
            redirect(request.url)

        else:

            textc = request.form['sentencea']
            textd = request.form['sentenceb']
            pred, score = predict_entft(textc, textd)
   
       

            return render_template("entailment_classification_result.html", sentencea=textc, pred= pred, score= score, sentenceb=textd)
    return render_template('ec.html' ,d=new_list, m= new_listmis ,e=counts, n=countf) 

In [44]:
@app.route('/wv', methods = ["POST", "GET"])

def wv():
    sone=[]
    sonemis=[]

    new_list =[]
    new_listmis =[]

    counts=0
    countf=0


   


    for (i) in range(30):
        
        trains1, trains2, label = train_df_features.iloc[i]
        sentence = trains1
        sentencea = trains2
       

        trains1, trains2 =prepare_datapoint(trains1,trains2)
      
        
        with graphv.as_default(): 
            pridiction = modelv.predict([trains1, trains2])[0]
        x =np.argmax(pridiction)
        r= (["neutral", "entailment", "contradiction"][np.argmax(pridiction)])
   
       
      
     
        
        if label==r:
            sone.append('sentence1: ' +  (sentence))
            sone.append('sentence2:  '+  (sentencea))
            sone.append('actual_label: '+(label))
            sone.append('pridicted_label: '+ r)
            counts+=1
        if label!=r:
            sonemis.append('sentence1: ' +  (sentence))
            sonemis.append('sentence2:  '+  (sentencea))
            sonemis.append('actual_label: '+(label))
            sonemis.append('pridicted_label: '+ r)
            countf+=1
                
        #new_list.append= [str(sone).split(maxsplit=0)]
        #print(len(sone))

        #new_list=[re.sub(',/n ', ' ', s.strip()) for s in sone]
        #new_list='[' + ' '(i for item in sone)+ ']'
        #new_list.append([string.replace((""","""), """ """) for string in sone])
        new_list=','.join(str(string) for string in sone)
        #new_list = codecs.decode(new_list, "unicode_escape",errors="replace")
        new_list = '<br>'.join(str(string) for string in sone)
        new_listmis=','.join(str(string) for string in sonemis)
        #new_list = codecs.decode(new_list, "unicode_escape",errors="replace")
        new_listmis = '<br>'.join(str(string) for string in sonemis)
   
    if request.method=='POST':
        if 'sentencea' not in request.form or 'sentenceb' not in request.form:
            flash('No sentence post')
            redirect(request.url)
        elif request.form['sentencea'] == '' or  request.form['sentenceb'] == '':
            flash('No sentence')
            redirect(request.url)
        else:

            textc = request.form['sentencea']
            textd = request.form['sentenceb']
            pred, score = predict_entwv(textc, textd)
           
            return render_template("entailment_classificationv_result.html", sentencea=textc, pred= pred, score= score, sentenceb=textd)
    return render_template('entailment_classificationv.html',d=new_list, m= new_listmis, e=counts, n=countf ) 

In [ ]:
if __name__ == "__main__":
    main()
    

    app.run()  


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [25/Nov/2020 16:44:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2020 16:44:34] "GET /static/vendor/animate.css/animate.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2020 16:44:34] "GET /static/vendor/owl.carousel/assets/owl.carousel.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2020 16:44:34] "GET /static/vendor/venobox/venobox.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2020 16:44:34] "GET /static/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2020 16:44:34] "GET /static/vendor/boxicons/css/boxicons.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2020 16:44:34] "GET /static/vendor/icofont/icofont.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2020 16:44:34] "GET /static/css/stylee.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2020 16:44:34] "GET /sta